<a href="https://colab.research.google.com/github/gshreya5/colab/blob/main/NLP_quora_insincere_tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quora Insincere Questions Classification


**GOAL** : Predict whether a question asked on Quora is sincere or not

# Load Libraries

In [1]:
import pandas as pd
import zipfile

# Load Dataset

In [3]:
!kaggle competitions download -c quora-insincere-questions-classification

100% 6.03G/6.03G [01:10<00:00, 153MB/s]
100% 6.03G/6.03G [01:10<00:00, 91.3MB/s]


In [4]:
path_to_zip_file = '/content/quora-insincere-questions-classification.zip'
directory_to_extract_to = '/content/quora-insincere-questions-classification'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [5]:
df = pd.read_csv('/content/quora-insincere-questions-classification/train.csv')

# Explore data

In [ ]:
df.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [ ]:
df.shape

(1306122, 3)

In [ ]:
df.columns

Index(['qid', 'question_text', 'target'], dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1306122 entries, 0 to 1306121
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   qid            1306122 non-null  object
 1   question_text  1306122 non-null  object
 2   target         1306122 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 29.9+ MB


In [ ]:
df.isnull().sum().sum()

0

In [ ]:
df.target.value_counts(normalize=True)

0    0.93813
1    0.06187
Name: target, dtype: float64

In [ ]:
df[df.target==1].question_text.values[:5]

array(['Has the United States become the largest dictatorship in the world?',
       'Which babies are more sweeter to their parents? Dark skin babies or light skin babies?',
       "If blacks support school choice and mandatory sentencing for criminals why don't they vote Republican?",
       'I am gay boy and I love my cousin (boy). He is sexy, but I dont know what to do. He is hot, and I want to see his di**. What should I do?',
       'Which races have the smallest penis?'], dtype=object)

# Implement Bag of Words

## Create vocabulary and Configure text prepocessing using Count Vectorizer

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
# text preprocessing

import nltk

# Step 1: word_tokenize
from nltk.tokenize import word_tokenize
nltk.download('punkt')
# Step 2: stop_words removal
from nltk.corpus import stopwords
nltk.download('stopwords')

stopwords = stopwords.words('english')
#Step 3: stemming
from nltk.stem import SnowballStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
def text_preprocessor(text):
  return [SnowballStemmer(language='english').stem(word) for word in word_tokenize(text) if word not in stopwords]

In [11]:
vectorizer = CountVectorizer(lowercase=True, tokenizer = text_preprocessor, max_features=1000 )
vectorizer.fit(df.question_text)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(max_features=1000,
                tokenizer=<function text_preprocessor at 0x7f90a2cf3a60>)

In [12]:
vectorizer.get_feature_names_out()

array(['!', '$', '%', '&', "'", "''", "'m", "'s", '(', ')', ',', '-', '.',
       '1', '10', '100', '12', '12th', '15', '2', '20', '2017', '2018',
       '3', '4', '5', '6', '7', '8', ':', '?', '[', ']', '``', 'abl',
       'abroad', 'abus', 'accept', 'access', 'accomplish', 'accord',
       'account', 'achiev', 'acid', 'act', 'action', 'activ', 'actor',
       'actual', 'ad', 'add', 'address', 'admiss', 'adult', 'advanc',
       'advantag', 'advic', 'affect', 'africa', 'african', 'age', 'ago',
       'air', 'allow', 'almost', 'alon', 'alreadi', 'also', 'altern',
       'alway', 'amazon', 'america', 'american', 'among', 'amount',
       'analysi', 'android', 'anim', 'anoth', 'answer', 'anyon', 'anyth',
       'apart', 'app', 'appear', 'appl', 'appli', 'applic', 'approach',
       'arab', 'area', 'arm', 'armi', 'around', 'art', 'asian', 'ask',
       'associ', 'atheist', 'attack', 'attend', 'attract', 'australia',
       'avail', 'averag', 'avoid', 'away', 'b', 'babi', 'back', 'bad',
  

## Transform text to vectors using Count Vectorizer

In [13]:
inputs = vectorizer.transform(df.question_text)

In [14]:
inputs.shape

(1306122, 1000)

In [15]:
inputs.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [16]:
test = pd.read_csv('/content/quora-insincere-questions-classification/test.csv')

In [17]:
test_inputs = vectorizer.transform(test.question_text)

# ML for Text Classification

### Split into train and test set

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
train_feature, test_feature, train_target, test_target =  train_test_split(inputs,df.target, test_size = 0.2, random_state=42)

In [20]:
train_feature.shape

(1044897, 1000)

### Train a logistic regression model

In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
clf = LogisticRegression(max_iter = 2000, solver='sag' ).fit(train_feature, train_target)

### Predict on test data

In [23]:
from sklearn.metrics import accuracy_score, f1_score

In [24]:
accuracy_score(test_target,clf.predict(test_feature) )

0.9462072925638817

In [25]:
f1_score(test_target,clf.predict(test_feature))

0.38009528851244045

# Submission

In [ ]:
pred = clf.predict(test_inputs)

In [ ]:
test['prediction'] = pred

In [ ]:
del test['question_text']

In [ ]:
test.to_csv('submission.csv', index=None)